# Iris flower classification with scikit-learn (run model explainer locally)

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/explain-tabular-data-local/explain-local-sklearn-multiclass-classification.png)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

Explain a model with the AML explain-model package

1. Train a SVM classification model using Scikit-learn
2. Run 'explain_model' with full data in local mode, which doesn't contact any Azure services
3. Run 'explain_model' with summarized data in local mode, which doesn't contact any Azure services
4. Visualize the global and local explanations with the visualization dashboard.

In [1]:
import sys
sys.version

'3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 18:50:55) [MSC v.1915 64 bit (AMD64)]'

In [2]:
#!pip install --upgrade azureml-sdk

In [3]:
#!pip install azureml-sdk[explain,contrib]

In [2]:
import azureml.core
print("Version Azure ML service :", azureml.core.VERSION)

Version Azure ML service : 1.0.53


In [3]:
from sklearn.datasets import load_iris
from sklearn import svm
from azureml.explain.model.tabular_explainer import TabularExplainer

C:\Anaconda\envs\AzureML\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# 1. Run model explainer locally with full data

## Load the breast cancer diagnosis data

In [4]:
iris = load_iris()
X = iris['data']
y = iris['target']
classes = iris['target_names']
feature_names = iris['feature_names']

In [5]:
feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [6]:
classes

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Train a SVM classification model, which you want to explain

In [8]:
clf = svm.SVC(gamma=0.001, C=100., probability=True)
model = clf.fit(x_train, y_train)

## Explain predictions on your local machine

In [9]:
tabular_explainer = TabularExplainer(model, x_train, features = feature_names, classes=classes)

C:\Anaconda\envs\AzureML\lib\site-packages\shap\explainers\linear.py:49: UserWarning: The default value for feature_dependence has been changed to "independent"!
  warnings.warn('The default value for feature_dependence has been changed to "independent"!')


## Explain overall model predictions (global explanation)

In [10]:
global_explanation = tabular_explainer.explain_global(x_test)

C:\Anaconda\envs\AzureML\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \


**Shap = SHapley Additive exPlanations**

**Shapley** values calculate the importance of a feature by comparing what a model predicts with and without the feature. However, since the order in which a model sees features can affect its predictions, this is done in every possible order, so that the features are fairly compared.

In [11]:
# Sorted SHAP values
print('Ranked global importance values: {}'.format(global_explanation.get_ranked_global_values()))
print()

# Corresponding feature names
print('Ranked global importance names: {}'.format(global_explanation.get_ranked_global_names()))
print()

# feature ranks (based on original order of features)
print('Global importance rank: {}'.format(global_explanation.global_importance_rank))
print()

# per class feature names
print('Ranked per class feature names: {}'.format(global_explanation.get_ranked_per_class_names()))
print()

# per class feature importance values
print('Ranked per class feature values: {}'.format(global_explanation.get_ranked_per_class_values()))

Ranked global importance values: [0.33272091943477605, 0.08628005286986912, 0.01734866350913634, 0.011047507408969781]

Ranked global importance names: ['petal length (cm)', 'petal width (cm)', 'sepal length (cm)', 'sepal width (cm)']

Global importance rank: [2, 3, 0, 1]

Ranked per class feature names: [['petal length (cm)', 'petal width (cm)', 'sepal width (cm)', 'sepal length (cm)'], ['petal length (cm)', 'petal width (cm)', 'sepal length (cm)', 'sepal width (cm)'], ['petal length (cm)', 'petal width (cm)', 'sepal length (cm)', 'sepal width (cm)']]

Ranked per class feature values: [[0.38659459709769806, 0.02599030683424756, 0.011487634888095268, 0.003528143360776975], [0.37436360331772967, 0.10945924732700968, 0.025737452416922876, 0.008786774156702992], [0.23720455788890035, 0.12339060444835012, 0.02278039474970917, 0.012868113182111087]]


In [12]:
dict(zip(global_explanation.get_ranked_global_names(), global_explanation.get_ranked_global_values()))

{'petal length (cm)': 0.33272091943477605,
 'petal width (cm)': 0.08628005286986912,
 'sepal length (cm)': 0.01734866350913634,
 'sepal width (cm)': 0.011047507408969781}

## Explain overall model predictions as a collection of local (instance-level) explanations

In [13]:
# feature shap values for all features and all data points in the training data
print('local importance values: {}'.format(global_explanation.local_importance_values))

local importance values: [[[-0.001616394175075031, -0.008115947388685596, -0.26754163990546226, -0.03176073978281818], [-0.0017934998503943977, -0.020619797023653325, -0.2749026860364282, -0.01073544865500864], [0.0017616462109611986, 0.034076149864299965, 0.5909356477536405, 0.032317541796518356], [-0.004415694973788992, -0.0051149975235454115, -0.2873221272003731, -0.020099960320811627], [0.0051218622287174664, 0.013764909080055399, 0.573757157980227, 0.04458952930563176], [-0.0027397576207472407, 0.00016526434142219437, -0.28059874101216287, -0.03360916726167329], [0.003840463692482976, 0.013733343933024056, 0.6036558793740525, 0.029455112395089578], [-0.004522832004131089, -0.002078518112959782, -0.29141918203995193, -0.014794264455640804], [-0.004877163058473161, -0.005574480702983914, -0.28868349403692073, -0.013792991191311], [-0.0021790123792854166, -0.008519420880326811, -0.27809364086551397, -0.016062339299479367], [-0.0015757092396594952, -0.008161279290908957, -0.2892964507

## Explain local data points (individual instances)

In [14]:
# explain the first member of the test set
instance_num = 0
local_explanation = tabular_explainer.explain_local(x_test[instance_num,:])

In [15]:
# get the prediction for the first member of the test set and explain why model made that prediction
prediction_value = clf.predict(x_test)[instance_num]

sorted_local_importance_values = local_explanation.get_ranked_local_values()[prediction_value]
sorted_local_importance_names = local_explanation.get_ranked_local_names()[prediction_value]

dict(zip(sorted_local_importance_names, sorted_local_importance_values))

{'petal width (cm)': 0.3201456817891125,
 'petal length (cm)': 0.2719503919774189,
 'sepal length (cm)': 0.009581681428778716,
 'sepal width (cm)': 0.005273730331263854}

## Load visualization dashboard

In [16]:
# Note you will need to have extensions enabled prior to jupyter kernel starting
!jupyter nbextension install --py --sys-prefix azureml.contrib.explain.model.visualize
!jupyter nbextension enable --py --sys-prefix azureml.contrib.explain.model.visualize
# Or, in Jupyter Labs, uncomment below
# jupyter labextension install @jupyter-widgets/jupyterlab-manager
# jupyter labextension install microsoft-mli-widget

C:\Anaconda\envs\AzureML\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Installing C:\Anaconda\envs\AzureML\lib\site-packages\azureml\contrib\explain\model\visualize\static -> microsoft-mli-widget
Out of date: C:\Anaconda\envs\AzureML\share\jupyter\nbextensions\microsoft-mli-widget\extension.js
Copying: C:\Anaconda\envs\AzureML\lib\site-packages\azureml\contrib\explain\model\visualize\static\extension.js -> C:\Anaconda\envs\AzureML\share\jupyter\nbextensions\microsoft-mli-widget\extension.js
Out of date: C:\Anaconda\envs\AzureML\share\jupyter\nbextensions\microsoft-mli-widget\extension.js.map
Copying: C:\Anaconda\envs\AzureML\lib\site-packages\azureml\contrib\explain\model\visualize\static\extension.js.map -> C:\Anaconda\envs\AzureML\share\jupyter\nbextensions\microsoft-mli-widget\extensi

In [17]:
from azureml.contrib.explain.model.visualize import ExplanationDashboard

In [18]:
ExplanationDashboard(global_explanation, model, x_test)

ExplanationWidget(value={'localExplanations': [[[-0.001616394175075031, -0.008115947388685596, -0.267541639905…